In [1]:
import re
import pandas as pd
import numpy as np
import csv

import pylsh

In [2]:
model_path = 'model/'
model_name = 'model-20170512-110547'
checkpoint_path = 'model-20170512-110547.ckpt-250000'
src_images = '../cropped_photos/'
IMG_SHAPE = (160, 160)


In [3]:
embeddings = pd.read_csv("data/cropped_embeddings.csv", names=['index', 'embedding'], dtype=str, skiprows=1)

In [4]:
embeddings['float_values'] = embeddings.embedding.apply(lambda x: re.sub(' +', ' ', str(x).replace('\n', ' ')
                                                                         ).replace('[', '') \
                                                                          .replace(']', '') \
                                                                          .split(' '))

In [5]:
embeddings['float_values'] = embeddings['float_values'].apply(lambda x: 
                                                        np.array(list(map(float, 
                                                                          [item for item in x if item != '']))))

In [6]:
len(embeddings.float_values[0])

128

In [7]:
# embeddings.float_values[0]

In [8]:
embeddings_list = embeddings.float_values.values.tolist()

In [9]:
len(embeddings.values)

157220

In [10]:
index = pylsh.PyLSH(50, 64, 128)

In [11]:
index.create_splits()

In [12]:
index.write_planes_to_file(b'./data/split.txt')

True

In [ ]:
for i, cur_emb in zip(embeddings.index, embeddings.float_values):
    index.add_to_table(i, cur_emb)
    if i % 10000 == 0:
        print(i)
#     if i == 10:
#         break

0
10000
20000
30000
40000


In [ ]:
# !mkdir ./data/index

In [ ]:
index.write_hash_tables_to_files(b'./data/index')

In [ ]:
index.write_index_embedding_dict(b'./data/index_embedding.txt')

А теперь найдем для них соседей

In [ ]:
bad = index.find_k_neighbors(10, embeddings.loc[0, 'float_values'])
bad

In [ ]:
good = index.dummy_k_neighbors(10, embeddings.index.values.tolist()[:10], embeddings_list[:10], \
                         embeddings.loc[0, 'float_values'])
good

In [ ]:
set(bad) & set(good)

In [ ]:
index.find_k_neighbors(10, embeddings.loc[1, 'float_values'])

In [ ]:
index.find_k_neighbors(10, embeddings.loc[2, 'float_values'])

In [29]:
index.calculate_distance(embeddings.loc[0, 'float_values'], embeddings.loc[1, 'float_values'])

0.1779484206681775